# Translation of specification

These are notes on how we translate a specification of an optimisation problem for MOLA into an abstract Pyomo model.



# Specification

This is the version 3 of the LP specification.

## Indices and sets

The index is a label that identifies an element in a set. For simplicity, we shall use the index to refer to the element that it indexes.

* $af \in AF$ is an index for an openLCA product flow imported from an openLCA database. This includes new flows that are defined by the user for the optimisation problem.
* $f\in F$ is an index for a user-defined flow.
* $f_m \in F_m \subset F$ is an index for a user-defined material flow (e.g. energy, material) to be considered in the optimisation problem.
* $f_s \in F_s \subset F$ is an index for an user-defined service flow (e.g. energy storage, transport) to be considered in the optimisation problem.
* $f_{t} \in F_{t} \subset F$ is an index for transport service flow i.e. transport mode \{road, train freight, air etc\}.
* $e \in E$ is an elementary flow index for elementary flows imported from an openLCA database. A system process in the openLCA database is by definition broken down into a set of these elementary flows.
* $ap \in AP$ is an index for a process in the set of all processes contained in an openLCA database. This include user-defined processes specifically designed for the optimisation tool.
* $p\in P\subset AP$ is a process index for processes that make up the optimisation problem.
* $t \in T$ is the time interval by time discretization $\{t_1, t_2, t_3, t_4 \ldots t_n\}$.
* $k \in K$ a task index $k$ in the set of all task indices $K$.
* $d \in D$ an index for a demand $d$ in the set of demand indices $D$. 
* $akpi \in AKPI$ is an indexes for all key performance indicators $KPI$ in an openLCA database. This includes key performance indicators defined by the user of the optimisation tool that must be added to the openLCA database.
* $kpi \in KPI\subset AKPI$ is an index that identifies those performance indicators that the user wish to use in the optimisation problem.

## Parameters

### User-defined

* $C_{f_m, k, d, t}$ Conversion factor for material flows to generate per unit of demand product/services $d$ at task $s$, time $t$. If not defined default value is 0.

* $D_{d,t}$ Demand for final product/service $d$ at time t; If not defined, default value is 0.

* $D_d^{total}$ Total demand for final product/service over the whole optimisation time period.

* $L_{f_m, f_s}$ Binary factor to link services flows to material flows e.g. energy storage, material storage, transport; If not defined, default value is 0.

* $X_{k, t}$ Longitude for where the material flow $f_m$ is transported in task $k$.

* $Y_{k, t}$ Latitude for where the material flow $f_m$ is transported in the task $k$.

* $d_{p,f_m, k, t}$ Total travel distance between process $p$ (where material flow $f_m$ is produced) and task $k$ (where material flow $f_m$ is transported to) at time $t$

$$
d_{p,f_m,k,t}=M(X_{k,t},Y_{k,t},X_{p, f_m}^I,Y^I_{p, f_m})
$$

where $M$ is a function that measures this distance e.g. Haversine - see http://www.movable-type.co.uk/scripts/latlong.html


* $\phi_{f, p, t}$ Cost co-efficient for material, service and transport flows $f$ produced by $p$ at time t.


### Imported from openLCA and model initialisation

* $Ef_{akpi, e}$ Environmental impact characterisation factor for elementary flow $e$ and performance indicator $akpi$.

* $EF_{e, f, p}$ Elementary flow $e$ to link with product flow $f$ through process $p$.

* $EI_{akpi, f, ap}$ Calculated environmental impact for product flow $𝑓$ through process $ap$ and performance indicator $kpi$.

* $X^I_{p,f_m}$ Longitude from the location table given in a process for material flow $f_m$.

* $Y^I_{p,f_m}$ Latitude from the location table given in the process for material flow $f_m$.

### Continuous variables


* $Obj_{kpi}$ Objective functions for the user-defined KPIs.
* $Flow_{f_m, k, t}$ is the flow of material input $f_m$ to task $k$ at time $t$.
* $S_{f_s, k, t}$ is the temporary storage of service input flow $f_s$ at task $k$ and time $t$. **Perhaps call it $Service$ like $Flow$.**
* $f_{f_m,f_t,k,t}$ Total quantity of materials $f_m$ transported through transport mode $f_t$ at task $k$ and time interval $t$ (unit: kg).
* $T_{f_t,k,t}$ Quantity times distance of all materials transported through the transport mode $f_t$ at task $k$ and time interval $t$ (unit: kg km).

## Objective function

Our objective is to minimise the environmental impact of elementary flows and the economic cost derived from a network of processes.

Consequently, the objective is for a fixed impact category $kpi$

$$
\min_D Obj_{kpi}
$$

and

$$
\min_D Obj_{cost}
$$

where the decision variables are defined by the set 

$$
D=\cup_{F,K,T}\{Flow_{f_m, k, t}, S_{f_s, k, t}, f_{f_m,f_t,k,t}, T_{f_t,k,t}\}
$$

The environmental impact is the sum of the environmental impacts arising from material, service flows, and transport flows:

$$
Obj_{kpi} = \sum_{f_m, k,t} Flow_{f_m, k, t}EI_{kpi, f_m, p} + 
\sum_{f_s, k ,t} S_{f_s, k, t}EI_{kpi, f_s, p} +
\sum_{f_t, k, t}T_{f_t, k,t}EI_{kpi,f_t,p}.
$$

The economic impact is the sum of the economic impacts arising from material, service and transport flows:

$$
Obj_{cost} = \sum_{f_m, k, t} Flow_{f_m, k, t}\phi_{f_m, p, t} +
\sum_{f_s, k, t} S_{f_s, k, t}\phi_{f_s, p, t} +
\sum_{f_t, k, t}T_{f_t, k, t}\phi_{f_t, p, t}
$$

Here the environmental impact of flow $f\in F$ measured by impact factor $kpi$ is 

$$
EI_{kpi, f, p} = \sum_e Ef_{kpi, e}EF_{e, f, p}
$$

where the flow $f$ is the product flow for the process $p\in P$. Here $Ef_{kpi, e}$ denotes the impact factor indexed by impact category $kpi$ and environmental flow $e$ and $EF_{e, f, p}$ is the amount of elementary flow generated by the product flow $f$ by process $p$. If $f\in F_m\cup F_s$ then the breakdown of flow into elementary flow amounts $EF_{e, f, p}$ must be calculated in openLCA by constructing a *system process*, which is then imported into the optimisation tool. Otherwise the flow is a product flow from an existing system process in openLCA so there already is a breakdown.

The quantity of service flow $S_{f_s,k,t}$ is assumed to be a weighted sum of the material flows at task $k$ and time $t$ i.e.

$$
S_{f_s,k,t} = \sum_{f_m} L_{f_m, f_s}Flow_{f_m, k, t},
$$

where $L_{f_m, f_s}$ is a parameter than links the service flow $f_s$ to the material flow $f_m$  in task $k$ at time $t$.

For any material flow $f_m$, the total quantity $Flow_{f_m,k,t}$ is dependent on the quantify of $f_m$ transported through transport mode $f_t$ at task $k$ and time interval $t$. The variable $T_{f_t,k,t}$ is defined by the quantity of $f_m$ transported via transport mode $f_t$ at task $k$ and time interval $t$ and the transport distance for shipping $f_m$ from initial production location $(X^I_{p,f_m},Y^I_{p,f_m})$ to final task location $X_{k,t}, Y_{k,t}$.

$$
Flow_{f_m,k,t}=\sum_{f_t} f_{f_m,f_t,k,t}
$$

$$
T_{f_t, k, t} = \sum_{p, f_m} f_{f_m, f_t, k, t}d_{p, f_m, k, t}
$$

## Constraints

The conversion of temporary storage to flow and demand requires

$$
Flow_{f_m, k, t} \geq \sum_d D_{d,t} C_{f_m, k, d, t} + S_{f_m, k, t} - S_{f_m, k, t-1}
$$

The total material flow minus final storage must satisfy the total demand over the time horizon so

$$
\sum_t Flow_{f_m, k, t}  \geq \sum_{d,t} D_d^{total} C_{f_m, k, t} + S_{f_m,k,t_n}
$$

Finally, we require

$$
S_{f_s, k, t} \geq 0
$$

$$
f_{f_m,f_t,k,t} \geq 0
$$


# Abstract Pyomo Model

The specification is translated into an abstract pyomo model.

In [1]:
from pyomo.environ import *
abstract_model = AbstractModel()

## Indices and sets

There is no instantiation of sets just placeholders for data. Some of the data must be supplied by the user via a GUI or programmatically and some must come from a data source.

### User-defined

These are placeholders for the GUI.

In [2]:
abstract_model.F = Set(doc='Flows to optimise')
abstract_model.F_m = Set(doc='Material flows to optimise')
abstract_model.F_s = Set(doc='Service flows to optimise')
abstract_model.F_t = Set(doc='Transport flows to optimise')
abstract_model.P = Set(doc='Processes in the optimisation problem')
abstract_model.T = Set(doc='Time intervals')
abstract_model.K = Set(doc='Tasks')
abstract_model.D = Set(doc='Demands')
abstract_model.KPI = Set(doc='Performance indicators for optimisation problem')

### DB DataPortal

These sets are populated by reference ids from openLCA.

In [3]:
abstract_model.AF = Set(doc='All flows in openLCA database')
abstract_model.E = Set(doc='Elementary Flows in OpenLCA database')
abstract_model.AP = Set(doc='All processes from in OpenLCA database')
abstract_model.AKPI = Set(doc='All key performance indicators in an openLCA database')

So to generate a new model instance in the GUI the user first needs to specify an openLCA database. This will populate dropdowns so the user can make choices. 

Let's use a DataPortal and the module `sqgenerator` to populate the sets. We shall also need names and categories which may need more complicated queries that are best built using a query builder in module `sqlgenerator`.

In [4]:
import mola.sqlgenerator as sq
olca_dp = DataPortal()
db_file = '/mnt/disk1/data/openlca/sqlite/system/CSV_juice_ecoinvent_36_apos_lci_20200206_20201029-102818.sqlite'
olca_dp.load(filename=db_file, using='sqlite3', query="SELECT REF_ID FROM TBL_FLOWS", set=abstract_model.AF)
model_instance = abstract_model.create_instance(olca_dp)
model_instance.AF.pprint()

AF : All flows in openLCA database
    Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any : 7033 : {'dcb70fd5-e305-4920-b68b-40697cc6a206', '0f440cc0-0f74-446d-99d6-8ff0e97a2444', 'e336eee7-148a-4d1c-8027-780cbfafa12b', 'e6551223-73b6-4289-b841-c5cdeb25abd9', '33b38ccb-593b-4b11-b965-10d747ba3556', 'afd6d670-bbb0-4625-9730-04088a5b035e', '70ef743b-3ed5-4a6d-b192-fb6d62378555', '66f50b33-fd62-4fdd-a373-c5b0de7de00d', '099b36ab-4c03-4587-87f4-2f81e337afb8', 'aa7cac3a-3625-41d4-bc54-33e2cf11ec46', '4d40d8e3-9bc7-4ab1-ac5c-4f4a76fda8e5', 'ada3ecfe-8244-4389-bcce-e83ca4f66e09', '81c4ba39-8a3f-4a43-97b4-401605bbebf5', '13d898ac-b9be-4723-a153-565e2a9144ac', '77357947-ccc5-438e-9996-95e65e1e1bce', '5e883a00-04e6-4d96-8dce-12d7117c6635', 'e8fc62ba-678e-4706-97d2-b79d83e227d5', '78c3efe4-421c-4d30-82e4-b97ac5124993', 'e2d58f90-9aac-4ac4-87f7-bbd6cfc358e3', '78eb1859-abd9-44c6-9ce3-f3b5b33d619c', '68b9b577-90cf-49e4-88bb-a55d80f3ac5d', '0d3

## Parameters

### User-defined

These are defined by the GUI so they are placeholders.

In [5]:
abstract_model.C = Param(abstract_model.F_m, abstract_model.K, abstract_model.D, abstract_model.T)
# etc

### DB DataPortal

We cannot just load these when the database is specified because they depend on user input. So we cannot load these via a DataPortal. They are likely determined in a model build phase after user definition.

In [6]:
abstract_model.Ef = Param(abstract_model.AKPI, abstract_model.E)
abstract_model.EF = Param(abstract_model.E, abstract_model.F, abstract_model.P)
abstract_model.EI = Param(abstract_model.AKPI, abstract_model.F, abstract_model.E)
abstract_model.XI = Param(abstract_model.P, abstract_model.F_m)
abstract_model.YI = Param(abstract_model.P, abstract_model.F_m)

## Decision variables

These can be defined in the abstract model.

In [7]:
abstract_model.Flow = Var(abstract_model.F_m, abstract_model.K, abstract_model.T)
abstract_model.Service = Var(abstract_model.F_s, abstract_model.K, abstract_model.T)
abstract_model.Transport = Var(abstract_model.F_m, abstract_model.F_t, abstract_model.K, abstract_model.T)
abstract_model.Transport_Service = Var(abstract_model.F_t, abstract_model.K, abstract_model.T)

## Objective

This needs to calculated at build time from information in the GUI.

In [8]:
# abstract_model.obj = Objective(expr=sum(abstract_model.Flow[f]*abstract_model.EI_Pm[process[f]] for f in abstract_model.F_m) +
#                         sum(abstract_model.T[f]*abstract_model.EI_Pt[process[f]] for f in abstract_model.Ft), sense=minimize)
# abstract_model.obj.pprint()

## Constraints

These need to determined at build time from information in the GUI.

# Class model

Use a function/class to define the pyomo abstract model into those sets and parameters that are populated by the user versus those populated from an openLCA database. We may need a class if we want to add functionality to the abstract model that is not in pyomo.

In [2]:
def get_abstract_model(name):
    name = name
    abstract_model = AbstractModel()

    # User defined sets
    abstract_model.F = Set(doc='Flows to optimise')
    abstract_model.F_m = Set(doc='Material flows to optimise')
    abstract_model.F_s = Set(doc='Service flows to optimise')
    abstract_model.F_t = Set(doc='Transport flows to optimise')
    abstract_model.P = Set(doc='Processes in the optimisation problem')
    abstract_model.T = Set(doc='Time intervals')
    abstract_model.K = Set(doc='Tasks')
    abstract_model.D = Set(doc='Demands')
    abstract_model.KPI = Set(doc='Performance indicators for optimisation problem')

    # DataPortal populated
    abstract_model.AF = Set(doc='All flows in openLCA database')
    abstract_model.E = Set(doc='Elementary Flows in OpenLCA database')
    abstract_model.AP = Set(doc='All processes from in OpenLCA database')
    abstract_model.AKPI = Set(doc='All key performance indicators in an openLCA database')

    # etc
    
    return abstract_model

These are functions/methods to populate an abstract model.

In [1]:
def get_user_data(abstract_model, olca_data_portal):
    model_instance = abstract_model.create_instance(olca_data_portal)
    # generate a gui pane
    # populate dropdowns and request what sets/parameters are missing in model_instance
    # fail if user data incomplete
    # return partially concrete model with complete user data
    return model_instance
    
# write gui-less function to programmatically generate partially concrete model
        
def build_concrete_model(model_instance):
    # build objectives and constraints
    return concrete_model

def solve_concrete_model(concrete_model, solver="glpk"):
    # return pyomo model object for results
    opt = SolverFactory(solver)
    results = opt.solve(concrete_model)
    return results
    
def output_model(pyomo_model_results):
    # long form data by interating over sets, params, vars
    1

Lemon Toy model